In [8]:
!pip install pandas-datareader

    100% |████████████████████████████████| 112kB 1.8MB/s ta 0:00:01
twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from tkinter import *
class Page(object):
    def __init__(self):
        pass
#build a input page    
    def InputPage(self,root):
        root.title('Finance SuperMan')
        #root.geometry('300*100')
        self.user_input=StringVar()
        self.user_ouput=StringVar()
        
        #self.stoklist=self.read_stocklist
        #CREATE ATTRIBUTES FOR THIS INTERFACE
        self.title_1=Label(root,text="Welcome to Finance SuperMAN!")
        self.entry_label=Label(root,text="Enter a company: ")
        self.entry_box=Entry(root,textvariable=self.user_input)
        self.confirmButton=Button(root,text="confirm",
                                  command=self.OutputPage)
        #LAYOUTS THE WIDGETS
        self.title_1.grid(row=0,sticky=W)
        self.entry_label.grid(row=3,column=0,sticky=W)
        self.entry_box.grid(row=3,column=3,columnspan=2)
        self.confirmButton.grid(row=16,column=4,sticky=E)
    
#build a outputpage        
    def OutputPage(self):
        from tkinter import messagebox
        output=Toplevel()
        output.title('in-time Information')
        output.geometry('300x1000')
        title_1=Label(output,text="Welcome to Finance SuperMAN!")
        title_1.grid(row=0,sticky=W)
        ref=self.read_stocklist()
        ipt=self.user_input.get().upper()
        if ipt not in ref:
            messagebox.showinfo('enter error','please enter the right ticket name')
        else:
            #option button
            Button_competitor=Button(output,text="competitor analysis",
                                     command=lambda:self.comp_stat_table(ipt))
            Button_sector=Button(output,text="sector analysis",
                                      command=lambda:self.get_sector(ipt))
            Button_experts_analysis=Button(output,text="Experts Analyst Report",
                                      command=lambda:self.get_analysis(ipt))
#             Button_competitor=Button(output,text="review competitor",
#                                      command=lambda:self.get_competitor(ipt))
#             Button_competitor=Button(output,text="review competitor",
#                                      command=lambda:self.get_competitor(ipt))
#             Button_competitor=Button(output,text="review competitor",
#                                      command=lambda:self.get_competitor(ipt))
#             Button_competitor=Button(output,text="review competitor",
#                                      command=lambda:self.get_competitor(ipt))
#             Button_competitor=Button(output,text="review competitor",
#                                      command=lambda:self.get_competitor(ipt))
#             Button_competitor=Button(output,text="review competitor",
#                                      command=lambda:self.get_competitor(ipt))
#             Button_competitor=Button(output,text="review competitor",
#                                      command=lambda:self.get_competitor(ipt))
#             Button_competitor=Button(output,text="review competitor",
#                                      command=lambda:self.get_competitor(ipt))
#             Button_competitor=Button(output,text="review competitor",
#                                      command=lambda:self.get_competitor(ipt))
#             Button_competitor=Button(output,text="review competitor",
#                                      command=lambda:self.get_competitor(ipt))
            Button_competitor.grid(row=5,column=0,sticky=W)
            Button_sector.grid(row=6,column=0,sticky=W)
            Button_experts_analysis.grid(row=7,column=0,sticky=W)
            
            

#read in the company list        
    def read_stocklist(self):
        import pandas as pd
        company=pd.DataFrame()
        company=pd.read_csv('companylist.csv',usecols=['Symbol'])
        stocklist=company.append(pd.read_csv('companylist-2.csv',usecols=['Symbol'],header=0))
        stocklist=company['Symbol'].tolist()
        return stocklist
                                 
#============================= stock analyst expectations =============================================
    def get_analysis(self,ticker):
        from IPython.display import display
        from yahoo_fin import stock_info as si
        import pandas as pd
        df = si.get_analysts_info(ticker)
        for i in list(df.keys()):
            display(pd.DataFrame(df.get(i)))

#============================ expected returns table ==================================================

#============================ stock basic information ==================================================

#============================= key technical plot ======================================================

#============================= 

    
#============================= sector analysis==========================================================        
    def get_sector(self,ticker):
        import requests
        from bs4 import BeautifulSoup
        url = 'https://finance.yahoo.com/quote/' + ticker + '/profile?p=' + ticker

        response = requests.get(url)
        results_page = BeautifulSoup(response.content, 'lxml')
        profile = results_page.find('div', class_='asset-profile-container')
        sector = profile.find_all('strong')[0].get_text()
        industry = profile.find_all('strong')[1].get_text()
        display('Your Sector: '+sector+'    Your Industry: '+industry)
        self.sector_performance()
    
    def sector_data(self,sector_page_link):
        import requests
        from bs4 import BeautifulSoup
        import re

        response = requests.get(sector_page_link)
        results_page = BeautifulSoup(response.content, 'lxml')

        market_cap = results_page.find('div', class_='tab-content-wrapper').find_all('span')[3].get_text()
        market_weight = results_page.find('div', class_='tab-content-wrapper').find_all('span')[5].get_text()

        no_space = r'\S+'
        data = list()
        for i in range(10):
            data.append(re.findall(no_space, results_page.find('div', class_='sec-fundamentals').find_all('td')[i].get_text()))

        pe_last_year = data[0][0]
        pe_this_year = data[1][0]
        ev = data[2][0]
        eps = data[3][0]
        eps_growth = data[4][0]
        revenue_growth = data[5][0]
        roe = data[6][0]
        roi = data[7][0]
        debt_to_equity = data[8][0]
        dividend = data[9][0]
        return market_cap, market_weight, ev, revenue_growth, eps_growth, eps, dividend, pe_last_year, pe_this_year, roe, roi, debt_to_equity

    def sector_performance(self):
        import requests
        from bs4 import BeautifulSoup
        import pandas as pd

        stats_list = list()
        sector_list = list()
        measure_list = list()

        url = "https://eresearch.fidelity.com/eresearch/goto/markets_sectors/landing.jhtml"

        results_page = requests.get(url)
        soup = BeautifulSoup(results_page.content, 'lxml')
        all_performance = soup.find_all('a', class_='heading1')

        for sectors in all_performance:
            name = sectors.find('strong').get_text()
            link = "https://eresearch.fidelity.com" + sectors.get('href')
            stats_list.append(self.sector_data(link))
            sector_list.append(name)

        measure_list = ['Market Capital', 'Market Weight', 'Enterprise Value', 'Revenue Growth (TTM)', 'EPS Growth (TTM)', 'EPS (TTM)', 'Dividend Yeild', 'Trailing P/E', 'This Year\'s Estimated P/E', 'ROE (TTM)', 'ROI (TTM)', 'Debt/Equity (TTM)']
        sector_table = pd.DataFrame(stats_list, index=sector_list, columns=measure_list)
        display(sector_table)
        
#========================competitor analysis======================================================================            
    def get_row(self,ticker,measure_list):
        from pandas_datareader import data as web
        a = self.get_financial_stats(ticker)
        row = list()
        row.append(web.DataReader(ticker,"yahoo").iloc[-1,3])
        for i in measure_list:
            if not i == "Price":
                appended = 0
                for j in a:
                    if j[0] == i:
                        row.append(j[1])
                        appended = 1
                if appended == 0:
                    row.append('N/A')   
        return row
    
    def comp_stat_table(self,ticker):
        import pandas as pd
        measure_list = ['Price','Market Cap (intraday)','Enterprise Value','Operating Margin','Quarterly Revenue Growth','Quarterly Earnings Growth','Diluted EPS','Trailing Annual Dividend Yield','5 Year Average Dividend Yield','Payout Ratio','Trailing P/E','Forward P/E','PEG Ratio (5 yr expected)','Return on Equity','Total Debt/Equity','Book Value Per Share','Beta (3Y Monthly)']
        r = self.get_competitors(ticker)
        r.insert(0, ticker)

        s_row = self.get_row(ticker,measure_list)
        a_row = self.get_row(r[1],measure_list)
        b_row = self.get_row(r[2],measure_list)
        c_row = self.get_row(r[3],measure_list)
        d_row = self.get_row(r[4],measure_list)

        comprehensive_list = ['Price', 'Market Cap', 'Enterprise Value', 'Operating Margin', 'Revenue Growth (Quarter)', 'Earnings Growth (Quarter)', 'EPS', 'Dividend Yield', '5-Year Avg Dividend Yield', 'Payout Ratio', 'Trailing P/E', 'Forward P/E', 'PEG', 'ROE (TTM)', 'Debt/Equity', 'Book Value/Share', 'Beta']

        stats_list = [s_row,a_row,b_row,c_row,d_row]
        final_table = pd.DataFrame(stats_list, index=r, columns=comprehensive_list)
    
        display(final_table)

    def get_competitors(self,stock):
        import requests
        from bs4 import BeautifulSoup
        url = 'https://www.nasdaq.com/symbol/' + stock + '/stock-comparison'

        response = requests.get(url)
        results_page = BeautifulSoup(response.content, 'lxml')
        table = results_page.find('div', class_='genTable thin')
        competitors = list()
        for i in range(4):
            competitors.append(table.find_all('input')[i].get('value'))
        return(competitors) 
        
    def get_financial_stats(self,ticker):
        import requests
        from bs4 import BeautifulSoup
        url = "https://finance.yahoo.com/quote/" + ticker + "/key-statistics?p=" + ticker

        response = requests.get(url)
        results_page = BeautifulSoup(response.content,'lxml')
        stats_lists = results_page.find_all('table')
        test_list = list()
        for i in stats_lists:
            stat_name = i.find_all('span')
            stock_stat = i.find_all('td',class_='Fz(s) Fw(500) Ta(end)')
            for j in range(len(stock_stat)):
                if not stat_name[j].get_text() == 'N/A':
                    test_list.append((stat_name[j].get_text(),stock_stat[j].get_text()))
        return test_list
    
    
    
    
root=Tk()
app=Page()
app.InputPage(root)
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/tkinter/__init__.py", line 1702, in __call__
    return self.func(*args)
  File "<ipython-input-2-815b23078df5>", line 44, in <lambda>
    command=lambda:self.get_analysis(ipt))
  File "<ipython-input-2-815b23078df5>", line 83, in get_analysis
    from yahoo_fin import stock_info as si
ModuleNotFoundError: No module named 'yahoo_fin'
